In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('contacts.json')

In [4]:
df_grouped = df[["Id","Email","Phone","OrderId"]].melt(id_vars="Id")

In [6]:
df_grouped.groupby("value").agg(lambda x: list(x))

,Id,variable
value,,
,"[1, 2, 4, 5, 10, 14, 15, 16, 25, 26, 32, 34, 3...","[Email, Email, Email, Email, Email, Email, Ema..."
000002957,[318092],[Phone]
00000837565,[411745],[Phone]
00001018140,[333273],[Phone]
0000109887,[170421],[Phone]
...,...,...
zzyYjbPWM@hotmail.com,"[143089, 338473]","[Email, Email]"
zzyacSRLb@gmail.com,[425119],[Email]
zzylmjkoqfCkTOxi@hotmail.com,[146556],[Email]


In [27]:
df_test = df[:]
df_test = df_test.replace(r'^\s*$', np.nan, regex=True)

In [28]:
#Email value check
df_email = df_test.drop(columns=['Phone','Contacts','OrderId'])
df_email = df_email.groupby(['Email']).agg(lambda x: list(x))
df_email = df_email[df_email['Id'].map(len) > 1]
df_email.rename(columns={"Id":"email_check"}, inplace=True)

In [29]:
df1 = df_test.merge(df_email, how ='left', on ='Email')

In [30]:
#Phone value check
df_phone = df_test.drop(columns=['Email','Contacts','OrderId'])
df_phone = df_phone.groupby(['Phone']).agg(lambda x: list(x))
df_phone = df_phone[df_phone['Id'].map(len) > 1]
df_phone.rename(columns={"Id":"phone_check"}, inplace=True)

In [31]:
df2 = df1.merge(df_phone, how ='left', on ='Phone')

In [32]:
#OrderId value Check
df_order = df_test.drop(columns=['Email','Contacts','Phone'])
df_order = df_order.groupby(['OrderId']).agg(lambda x: list(x))
df_order = df_order[df_order['Id'].map(len) > 1]
df_order.rename(columns={"Id":"order_check"}, inplace=True)

In [33]:
df3 = df2.merge(df_order, how ='left', on ='OrderId')

In [34]:
df3['email_check'] = df3['email_check'].fillna("").apply(list)
df3['phone_check'] = df3['phone_check'].fillna("").apply(list)
df3['order_check'] = df3['order_check'].fillna("").apply(list)
df3['id_check'] = df3['email_check'] + df3['order_check'] + df3['phone_check']

In [35]:
df3 = df3.drop(columns=['phone_check','email_check','order_check'])

In [36]:
def indirect_check(row):
    newlist = []
    for i in row['id_check']:
        newlist = newlist + df3['id_check'][i]

def additional_indirect_check(row):
    newlist = []
    for i in row['id_check_ans']:
        newlist = newlist + df3['id_check_ans'][i]
    return newlist

In [77]:
for row in df3.head().itertuples():
    print(row.id_check)

[]
[1, 2458, 476346]
[2, 348955]
[]
[]


In [82]:
def indirect_check(row):
    newlist = []
    for i in row['id_check']:
        newlist = newlist + df3['id_check'][i]

In [ ]:
def itertuples_impl(df):
  cutoff_date = datetime.date.today() + datetime.timedelta(days=2)
  return pd.Series(
    eisenhower_action(
      row.priority == 'HIGH', row.due_date <= cutoff_date)
    for row in df.itertuples()
  )

In [ ]:
def itertuples_indirectcheck(df):
  return pd.Series(
    newlist = []
    for i in row['id_check']:
        newlist = newlist + df3['id_check'][i]
    return newlist
    for row in df.itertuples()
  )

In [37]:
%timeit df3['id_check_ans'] = df3.apply(indirect_check, axis=1)

9.27 s ± 75.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:
df3

,Id,Email,Phone,Contacts,OrderId,id_check,id_check_ans
0,0,gkzAbIy@qq.com,NaN,1,NaN,[],[]
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"[1, 2458, 476346]","[1, 2458, 476346, 2458, 98519, 1, 2458, 476346..."
2,2,NaN,9125983679,0,NaN,"[2, 348955]","[2, 348955, 159312, 322639, 348955, 2, 348955]"
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,[],[]
4,4,NaN,300364407,2,NaN,[],[]
...,...,...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv,[],[]
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS,[],[]
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO,[],[]
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku,"[121111, 499998]","[121111, 499998, 121111, 499998]"


In [133]:
for t in range(len(df3.head())):
    for row in df3.head()["id_check"][t]: 
        print(row)

1
2458
476346
2
348955


In [124]:
[[[row.id_check[t] for t,d in enumerate(row.id_check)] + []] for row in df3.head()]

AttributeError: 'str' object has no attribute 'id_check'

In [16]:
def remove_duplicates_and_sort(row):
    return sorted(list(dict.fromkeys(row['id_check_ans'])))

df3['id_check_ans'] = df3.apply(remove_duplicates_and_sort, axis=1)
df3['id_check_ans'] = df3.apply(additional_indirect_check, axis=1)

In [18]:
def remove_duplicates_and_sort(row):
    return sorted(list(dict.fromkeys(row['id_check_ans'])))

In [19]:
df3['id_check_ans'] = df3.apply(remove_duplicates_and_sort, axis=1)

In [ ]:
def single_ticket_correction(row):
    if len(row['id_check']) == 0:
        return [row['Id']]
    else:
        return row['id_check_ans']

In [ ]:
df3['id_check_ans'] = df3.apply(single_ticket_correction, axis=1)

In [ ]:
def format_answer(row):
    answer = ''
    for i in (row['id_check_ans']):
        answer = answer + '-' + str(i)
    return answer[1:]

In [ ]:
df3['ticket_trace'] = df3.apply(format_answer, axis=1)

In [ ]:
def calculate_contacts(row):
    contact = 0
    for i in row['id_check_ans']:
        contact = contact + df3['Contacts'][i]
    return contact

In [ ]:
df3['calculated_contacts'] = df3.apply(calculate_contacts, axis=1)

In [ ]:
df3['ticket_trace/contact'] = df3['ticket_trace'] + "," + df3['calculated_contacts'].apply(str)

In [ ]:
df3 = df3.set_index('Id')

In [ ]:
df3 = df3.drop(columns=['id_check','Email','Phone','Contacts','OrderId','id_check','id_check_ans','ticket_trace','calculated_contacts'])

In [ ]:
df3.to_csv('answer.csv')

In [3]:
df_selected = df[["Id","Email","Phone","OrderId"]]

In [4]:
df_long = df_selected.melt(id_vars="Id")

In [5]:
df_long

,Id,variable,value
0,0,Email,gkzAbIy@qq.com
1,1,Email,
2,2,Email,
3,3,Email,mdllpYmE@gmail.com
4,4,Email,
...,...,...,...
1499995,499995,OrderId,whJlUOGNqjxCRzmIRdURQvlNv
1499996,499996,OrderId,JqIXOONvrwvJxZqNxCYHqnrKS
1499997,499997,OrderId,beXCZSzcHaBwAYoDcpQqjuAFO
1499998,499998,OrderId,ehjeFACGiwrERQxbziMxwOWku


In [6]:
df_long_grouped = df_long.groupby("value").agg(lambda x: list(x))

In [107]:
[[row.Id[t] + for t,d in enumerate(row.Id)] for row in df_long_grouped[1:].head().itertuples()]

[[318092], [411745], [333273], [170421], [229812, 265783]]

In [7]:
my_list = list(df_long_grouped["Id"][1:])

In [136]:
d = {} 
for lst in my_list: 
    d.setdefault(lst[-1], []).extend(lst)    

<built-in method values of dict object at 0x265522F8>


In [138]:
sorted_dict = {k: d[k] for k in sorted(d)}

In [9]:
l = my_list

In [10]:
import networkx 
from networkx.algorithms.components.connected import connected_components

def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # each sublist is a bunch of nodes
        G.add_nodes_from(part)
        # it also imlies a number of edges:
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

G = to_graph(l)
connected_list = [list(i) for i in connected_components(G)]

In [38]:
temp_df = pd.DataFrame({'A':connected_list,
'B':connected_list})

In [40]:
temp_df.explode('A').sort_values('A')

,A,B
258167,0,[0]
13330,1,"[1, 165605, 2458, 140081, 115061, 98519, 476346]"
101946,2,"[159312, 2, 348955, 322639]"
245576,3,[3]
53992,4,[4]
...,...,...
18969,499995,[499995]
97669,499996,[499996]
79193,499997,[499997]
159285,499998,"[499998, 121111]"


In [23]:
L = [[1,4,6],[3,41],[41,78],[6,22,30]]

In [15]:
len_l = len(l)
i = 0
while i < (len_l - 1):
    for j in range(i + 1, len_l):

        # i,j iterate over all pairs of l's elements including new 
        # elements from merged pairs. We use len_l because len(l)
        # may change as we iterate
        i_set = set(l[i])
        j_set = set(l[j])

        if len(i_set.intersection(j_set)) > 0:
            # Remove these two from list
            l.pop(j)
            l.pop(i)

            # Merge them and append to the orig. list
            ij_union = list(i_set.union(j_set))
            l.append(ij_union)

            # len(l) has changed
            len_l -= 1

            # adjust 'i' because elements shifted
            i -= 1

            # abort inner loop, continue with next l[i]
            break

    i += 1

In [29]:
LL = set(itertools.chain.from_iterable(L)) 

for each in LL:
  components = [x for x in L if each in x]
  for i in components:
    L.remove(i)
  L += [list(set(itertools.chain.from_iterable(components)))]

# then L = [['k'], ['a', 'c', 'b', 'e', 'd', 'g', 'f', 'o', 'p']]

In [37]:
L = my_list